In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append("../../fastai/") # go to parent dir

# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

PATH = '/your_path/data/text'

In [8]:
sz=224
arch=resnext50
bs=28

tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs)

learn = ConvLearner.pretrained(arch, data)

learn.load('224_rexnext_50')

In [9]:
# Some Helper functions
from PIL import Image
def image_loader(path, expand_dim=False):
    img = Image.open(path)
    img = img.resize((sz, sz))
    img = np.array(img, dtype=np.float32)
    img = np.einsum('ijk->kij', img)
    if expand_dim:
        img = img[None]
    # convert to torch 
    img = torch.from_numpy(img)
    return img 

def return_sequential(layer_num, model):
    return nn.Sequential(
            *list(model.children())[:layer_num]
        )

class get_activation_layer(nn.Module):
    def __init__(self, model, total_layers):
        super().__init__()
        self.model = model
        self.total_layers = total_layers
        self.layer_models = []
        for i in range(self.total_layers):
             self.layer_models.append(return_sequential(i, self.model))
    def forward(self, x):
        self.outputs = []
#         print(x.shape)
        for i in range(self.total_layers):
            self.outputs.append(self.layer_models[i](x))
#             print('shape outputs layer_models[](x) ->',shape(self.outputs[i]))
#             print('layer_models[i]', i,'->',self.layer_models[i])
        return self.outputs


In [10]:
img_name = 'ittention2.jpg';
img_path = f'{PATH}/valid/text/{img_name}'
i = image_loader(img_path, expand_dim=True)
i = i.cuda()

# get actions for the first nr_layers(max:10) layers - the 11th is already FC / flatten
nr_layers = 6
tmp_model = get_activation_layer(model, nr_layers)

In [11]:
layer_outputs = tmp_model(Variable(i))

In [ ]:
# for index, layer in enumerate(layer_outputs):
k = 1 # the number of the layer from 0 to 10
j = 1 # the number of the filter between 0 and 2, or between 0 and 63, etc
x = layer_outputs[k].data[0][j]
# print(shape(x))
# print('-', shape(layer_outputs[k].data[0]))
# imshow(x)

import pathlib

pathlib.Path(f'{PATH}/layer_op/{img_name}/').mkdir(parents=True, exist_ok=True)

for k in range(10):
    for blocks in layer_outputs[k].data:
    #     print(shape(blocks))
        for key,block in enumerate(blocks):
            plt.figure()
            plt.title(key)
            plt.imshow(block) 
#             plt.savefig(f'{PATH}/layer_op/{img_name}/layer_{k}_filter{key}.jpg', dpi=1000)